In [1]:
import pandas as pd
import unidecode
import re
import nltk
import gensim
import numpy as np
import keras
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pickle
from keras.utils import np_utils
from keras.layers import Embedding
from keras.models import Sequential

/home/barbabra/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Data

In [2]:
paths = {
    "dev":"../data/macmorpho-v3/macmorpho-dev.txt",
    "test":"../data/macmorpho-v3/macmorpho-test.txt",
    "train":"../data/macmorpho-v3/macmorpho-train.txt",
}

In [3]:
#load data
sentences = []
for p in paths.values():
    file = open(p, "r")
    sentences = sentences + file.readlines()
    file.close()
data = pd.DataFrame()
data['sentences'] = sentences

In [4]:
len(data)

49932

In [5]:
data.head()

,sentences
0,"Ainda_ADV em_PREP dezembro_N de_PREP 1990_N ,_..."
1,"Porém_KC ,_PU como_KS a_ART previsão_N indica_..."
2,"""_PU O_ART crescimento_N é_V expressivo_ADJ ma..."
3,O_ART programa_N atende_V ainda_ADV as_ART cul...
4,"de_PREP qualquer_PROADJ maneira_N ,_PU toda_PR..."


# Pre-processing

vou fazer uma rede que recebe uma palavra e retorna 

- remover acentos
- remover pontuação (?) -> nao vou fazer isso... pq tbm é uma tag
- remover \n
- aplicar lower case
- get TAGS
- DEPOIS fazer isso:
- aplicar token
- fazer stemming
- aplicar pharese

- preparacao dos dados 
    separar as palavras e a tag. Dicionario tag: word
- create embedding com todos os dados
- olhar tamanho das palavras
- vetorzacao, padding... etc.

In [6]:
def remove_accents(sentence):
    return unidecode.unidecode(sentence)

In [7]:
def remove_character(sentence):
    return re.sub(' +',' ', (sentence.replace("\n", " ")).replace("\t", " "))

In [8]:
def pre_processing(sentence):
    sentence = remove_character(sentence)
    sentence = remove_accents(sentence)
    sentence = sentence.lower()
    return sentence

In [9]:
#pre-processing
data["sentences"] = data["sentences"].map(pre_processing)

In [10]:
#get text
data["corpus"] = data.loc[:, "sentences"].apply(lambda x : (" ".join([y.split("_")[0] for y in x.strip().split(" ")])).strip())

In [11]:
# get tags
data["tags"] = data.loc[:, "sentences"].apply(lambda x : (" ".join([y.split("_")[1] for y in x.strip().split(" ")])).strip())

In [12]:
data.head()

,sentences,corpus,tags
0,"ainda_adv em_prep dezembro_n de_prep 1990_n ,_...","ainda em dezembro de 1990 , foi editada a famo...",adv prep n prep n pu v pcp art adj n pu pro-ks...
1,"porem_kc ,_pu como_ks a_art previsao_n indica_...","porem , como a previsao indica entrada de fren...",kc pu ks art n v n prep n adj pu v propess v p...
2,"""_pu o_art crescimento_n e_v expressivo_adj ma...",""" o crescimento e expressivo mas , mesmo assim...",pu art n v adj kc pu pden pden pu v prep prep+...
3,o_art programa_n atende_v ainda_adv as_art cul...,o programa atende ainda as culturas de feijao ...,art n v adv art n prep n pu adj kc adj n pu pu...
4,"de_prep qualquer_proadj maneira_n ,_pu toda_pr...","de qualquer maneira , toda essa "" informalidad...",prep proadj n pu proadj proadj pu n pu prep+ar...


In [13]:
#apply token
data["corpus"] = data.loc[:, "corpus"].apply(lambda x : nltk.word_tokenize(x))
data["tags"] = data.loc[:, "tags"].apply(lambda x : nltk.word_tokenize(x))

In [14]:
data.head()

,sentences,corpus,tags
0,"ainda_adv em_prep dezembro_n de_prep 1990_n ,_...","[ainda, em, dezembro, de, 1990, ,, foi, editad...","[adv, prep, n, prep, n, pu, v, pcp, art, adj, ..."
1,"porem_kc ,_pu como_ks a_art previsao_n indica_...","[porem, ,, como, a, previsao, indica, entrada,...","[kc, pu, ks, art, n, v, n, prep, n, adj, pu, v..."
2,"""_pu o_art crescimento_n e_v expressivo_adj ma...","[``, o, crescimento, e, expressivo, mas, ,, me...","[pu, art, n, v, adj, kc, pu, pden, pden, pu, v..."
3,o_art programa_n atende_v ainda_adv as_art cul...,"[o, programa, atende, ainda, as, culturas, de,...","[art, n, v, adv, art, n, prep, n, pu, adj, kc,..."
4,"de_prep qualquer_proadj maneira_n ,_pu toda_pr...","[de, qualquer, maneira, ,, toda, essa, ``, inf...","[prep, proadj, n, pu, proadj, proadj, pu, n, p..."


# Embedding

In [15]:
data.columns

Index(['sentences', 'corpus', 'tags'], dtype='object')

## Phrases and Phraser

In [16]:
corpus = data.corpus.values
bigram_corpus_phrases = gensim.models.Phrases(corpus)
bigram_corpus = bigram_corpus_phrases[corpus]

/home/barbabra/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [17]:
size_emb = 300
model_word2vec = gensim.models.word2vec.Word2Vec(bigram_corpus, min_count=1, workers=4, seed=123, size=size_emb)

/home/barbabra/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [18]:
len(model_word2vec.wv.vocab)

56252

In [19]:
def create_dict_embedding(word2vec):
    embeddings_dict = {}
    #get size dimension
    size_vec = word2vec.wv.vector_size
    #create disctionary
    for word in list(word2vec.wv.vocab.keys()):
        embeddings_dict[word] = word2vec.wv[word] 
    return embeddings_dict

In [20]:
def get_embeddings(word2vec):
    embeddings_dict = create_dict_embedding(word2vec)
    size = len(embeddings_dict)
    embedding_matrix = np.ndarray((size, word2vec.wv.vector_size), dtype='float32')
    wv_map = {}
    pos = 0
    for i, (word, vector) in enumerate(embeddings_dict.items()):
        #pos = i + 1
        pos = i
        wv_map[word] = pos
        embedding_matrix[pos] = vector
    #pos += 1
    #wv_map["<unk>"] = pos
    #embedding_matrix[pos] = numpy.random.uniform(low=-0.05, high=0.05, size=dim)
    return embedding_matrix, wv_map

# Split data: Train, Test and Validation

In [21]:
data = shuffle(data, random_state=0)

In [22]:
corpus = data.corpus.values
tags = data.tags.values

In [23]:
X_train, X_test, y_train, y_test = train_test_split(corpus, tags, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val   = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

# Preparing data
- padding max_sentence_length  with 0
- index words
- index tags

In [24]:
embeddings, words_index = get_embeddings(model_word2vec)

In [25]:
tags_index = {}
i = 1
for tags in data.tags.values:
    for tag in tags:
        if(tag not in tags_index.keys()):
            tags_index[tag] = i
            i = i + 1

In [26]:
max_sentence_length = int(data["corpus"].map(len).describe()['max'] + 1)
max_sentence_length

249

## Index

In [212]:
from keras.preprocessing.sequence import pad_sequences

In [27]:
# SET sentence index and padding
def index_words(sentences, max_sentence_length, words_index):    
    print("indexing sentences...")
    text_max = False
    sent_words = []

    sentences_words_indexed = np.zeros((len(sentences), max_sentence_length), dtype='int32')

    for i, sentence in enumerate(sentences):
        print(str(i)+"/"+str(len(sentences)))
        text_max = False
        sent_words = []
        for word in sentence:
            if(word not in set(words_index.keys())):
                continue
            sent_words.append(words_index[word])
            if(len(sent_words) >= max_sentence_length):
                text_max = True
                break
        if(text_max == False):
            sent_words = np.pad(sent_words, (0, max_sentence_length - len(sent_words) % max_sentence_length), 'constant')
        sentences_words_indexed[i] = sent_words
    return sentences_words_indexed

In [28]:
len(X_train)

31956

In [29]:
len(X_train[:200])

200

In [210]:
X_train_inx = pickle.load(open("data/X_train_index.p", "rb"))
y_train_inx = pickle.load(open("data/y_train_index.p", "rb"))

X_test_inx = pickle.load(open("data/X_test_index.p", "rb"))
y_test_inx = pickle.load(open("data/y_test_index.p", "rb"))

X_val_inx = pickle.load(open("data/X_val_index.p", "rb"))
y_val_inx = pickle.load(open("data/y_val_index.p", "rb"))


In [30]:
#train
print("traing")
X_train_inx = index_words(X_train, max_sentence_length, words_index)
y_train_inx = index_words(y_train, max_sentence_length, tags_index)

pickle.dump(X_train_inx, open( "data/X_train_index.p", "wb" ) )
pickle.dump(y_train_inx, open( "data/y_train_index.p", "wb" ) )

#test
print("test")
X_test_inx = index_words(X_test, max_sentence_length, words_index)
y_test_inx = index_words(y_test, max_sentence_length, tags_index)

pickle.dump(X_test_inx, open( "data/X_test_index.p", "wb" ) )
pickle.dump(y_test_inx, open( "data/y_test_index.p", "wb" ) )

#validaotion
print("validation")
X_val_inx = index_words(X_val, max_sentence_length, words_index)
y_val_inx = index_words(y_val, max_sentence_length, tags_index)

pickle.dump(X_val_inx, open( "data/X_val_index.p", "wb" ) )
pickle.dump(y_val_inx, open( "data/y_val_index.p", "wb" ) )

traing
indexing sentences...
0/31956
1/31956
2/31956
3/31956
4/31956
5/31956
6/31956
7/31956
8/31956
9/31956
10/31956
11/31956
12/31956
13/31956
14/31956
15/31956
16/31956
17/31956
18/31956
19/31956
20/31956
21/31956
22/31956
23/31956
24/31956
25/31956
26/31956
27/31956
28/31956
29/31956
30/31956
31/31956
32/31956
33/31956
34/31956
35/31956
36/31956
37/31956
38/31956
39/31956
40/31956
41/31956
42/31956
43/31956
44/31956
45/31956
46/31956
47/31956
48/31956
49/31956
50/31956
51/31956
52/31956
53/31956
54/31956
55/31956
56/31956
57/31956
58/31956
59/31956
60/31956
61/31956
62/31956
63/31956
64/31956
65/31956
66/31956
67/31956
68/31956
69/31956
70/31956
71/31956
72/31956
73/31956
74/31956
75/31956
76/31956
77/31956
78/31956
79/31956
80/31956
81/31956
82/31956
83/31956
84/31956
85/31956
86/31956
87/31956
88/31956
89/31956
90/31956
91/31956
92/31956
93/31956
94/31956
95/31956
96/31956
97/31956
98/31956
99/31956
100/31956
101/31956
102/31956
103/31956
104/31956
105/31956
106/31956
107/31956
1

828/31956
829/31956
830/31956
831/31956
832/31956
833/31956
834/31956
835/31956
836/31956
837/31956
838/31956
839/31956
840/31956
841/31956
842/31956
843/31956
844/31956
845/31956
846/31956
847/31956
848/31956
849/31956
850/31956
851/31956
852/31956
853/31956
854/31956
855/31956
856/31956
857/31956
858/31956
859/31956
860/31956
861/31956
862/31956
863/31956
864/31956
865/31956
866/31956
867/31956
868/31956
869/31956
870/31956
871/31956
872/31956
873/31956
874/31956
875/31956
876/31956
877/31956
878/31956
879/31956
880/31956
881/31956
882/31956
883/31956
884/31956
885/31956
886/31956
887/31956
888/31956
889/31956
890/31956
891/31956
892/31956
893/31956
894/31956
895/31956
896/31956
897/31956
898/31956
899/31956
900/31956
901/31956
902/31956
903/31956
904/31956
905/31956
906/31956
907/31956
908/31956
909/31956
910/31956
911/31956
912/31956
913/31956
914/31956
915/31956
916/31956
917/31956
918/31956
919/31956
920/31956
921/31956
922/31956
923/31956
924/31956
925/31956
926/31956
927/31956


1594/31956
1595/31956
1596/31956
1597/31956
1598/31956
1599/31956
1600/31956
1601/31956
1602/31956
1603/31956
1604/31956
1605/31956
1606/31956
1607/31956
1608/31956
1609/31956
1610/31956
1611/31956
1612/31956
1613/31956
1614/31956
1615/31956
1616/31956
1617/31956
1618/31956
1619/31956
1620/31956
1621/31956
1622/31956
1623/31956
1624/31956
1625/31956
1626/31956
1627/31956
1628/31956
1629/31956
1630/31956
1631/31956
1632/31956
1633/31956
1634/31956
1635/31956
1636/31956
1637/31956
1638/31956
1639/31956
1640/31956
1641/31956
1642/31956
1643/31956
1644/31956
1645/31956
1646/31956
1647/31956
1648/31956
1649/31956
1650/31956
1651/31956
1652/31956
1653/31956
1654/31956
1655/31956
1656/31956
1657/31956
1658/31956
1659/31956
1660/31956
1661/31956
1662/31956
1663/31956
1664/31956
1665/31956
1666/31956
1667/31956
1668/31956
1669/31956
1670/31956
1671/31956
1672/31956
1673/31956
1674/31956
1675/31956
1676/31956
1677/31956
1678/31956
1679/31956
1680/31956
1681/31956
1682/31956
1683/31956
1684/31956

2339/31956
2340/31956
2341/31956
2342/31956
2343/31956
2344/31956
2345/31956
2346/31956
2347/31956
2348/31956
2349/31956
2350/31956
2351/31956
2352/31956
2353/31956
2354/31956
2355/31956
2356/31956
2357/31956
2358/31956
2359/31956
2360/31956
2361/31956
2362/31956
2363/31956
2364/31956
2365/31956
2366/31956
2367/31956
2368/31956
2369/31956
2370/31956
2371/31956
2372/31956
2373/31956
2374/31956
2375/31956
2376/31956
2377/31956
2378/31956
2379/31956
2380/31956
2381/31956
2382/31956
2383/31956
2384/31956
2385/31956
2386/31956
2387/31956
2388/31956
2389/31956
2390/31956
2391/31956
2392/31956
2393/31956
2394/31956
2395/31956
2396/31956
2397/31956
2398/31956
2399/31956
2400/31956
2401/31956
2402/31956
2403/31956
2404/31956
2405/31956
2406/31956
2407/31956
2408/31956
2409/31956
2410/31956
2411/31956
2412/31956
2413/31956
2414/31956
2415/31956
2416/31956
2417/31956
2418/31956
2419/31956
2420/31956
2421/31956
2422/31956
2423/31956
2424/31956
2425/31956
2426/31956
2427/31956
2428/31956
2429/31956

3084/31956
3085/31956
3086/31956
3087/31956
3088/31956
3089/31956
3090/31956
3091/31956
3092/31956
3093/31956
3094/31956
3095/31956
3096/31956
3097/31956
3098/31956
3099/31956
3100/31956
3101/31956
3102/31956
3103/31956
3104/31956
3105/31956
3106/31956
3107/31956
3108/31956
3109/31956
3110/31956
3111/31956
3112/31956
3113/31956
3114/31956
3115/31956
3116/31956
3117/31956
3118/31956
3119/31956
3120/31956
3121/31956
3122/31956
3123/31956
3124/31956
3125/31956
3126/31956
3127/31956
3128/31956
3129/31956
3130/31956
3131/31956
3132/31956
3133/31956
3134/31956
3135/31956
3136/31956
3137/31956
3138/31956
3139/31956
3140/31956
3141/31956
3142/31956
3143/31956
3144/31956
3145/31956
3146/31956
3147/31956
3148/31956
3149/31956
3150/31956
3151/31956
3152/31956
3153/31956
3154/31956
3155/31956
3156/31956
3157/31956
3158/31956
3159/31956
3160/31956
3161/31956
3162/31956
3163/31956
3164/31956
3165/31956
3166/31956
3167/31956
3168/31956
3169/31956
3170/31956
3171/31956
3172/31956
3173/31956
3174/31956

...

3034/7989
3035/7989
3036/7989
3037/7989
3038/7989
3039/7989
3040/7989
3041/7989
3042/7989
3043/7989
3044/7989
3045/7989
3046/7989
3047/7989
3048/7989
3049/7989
3050/7989
3051/7989
3052/7989
3053/7989
3054/7989
3055/7989
3056/7989
3057/7989
3058/7989
3059/7989
3060/7989
3061/7989
3062/7989
3063/7989
3064/7989
3065/7989
3066/7989
3067/7989
3068/7989
3069/7989
3070/7989
3071/7989
3072/7989
3073/7989
3074/7989
3075/7989
3076/7989
3077/7989
3078/7989
3079/7989
3080/7989
3081/7989
3082/7989
3083/7989
3084/7989
3085/7989
3086/7989
3087/7989
3088/7989
3089/7989
3090/7989
3091/7989
3092/7989
3093/7989
3094/7989
3095/7989
3096/7989
3097/7989
3098/7989
3099/7989
3100/7989
3101/7989
3102/7989
3103/7989
3104/7989
3105/7989
3106/7989
3107/7989
3108/7989
3109/7989
3110/7989
3111/7989
3112/7989
3113/7989
3114/7989
3115/7989
3116/7989
3117/7989
3118/7989
3119/7989
3120/7989
3121/7989
3122/7989
3123/7989
3124/7989
3125/7989
3126/7989
3127/7989
3128/7989
3129/7989
3130/7989
3131/7989
3132/7989
3133/7989


4231/7989
4232/7989
4233/7989
4234/7989
4235/7989
4236/7989
4237/7989
4238/7989
4239/7989
4240/7989
4241/7989
4242/7989
4243/7989
4244/7989
4245/7989
4246/7989
4247/7989
4248/7989
4249/7989
4250/7989
4251/7989
4252/7989
4253/7989
4254/7989
4255/7989
4256/7989
4257/7989
4258/7989
4259/7989
4260/7989
4261/7989
4262/7989
4263/7989
4264/7989
4265/7989
4266/7989
4267/7989
4268/7989
4269/7989
4270/7989
4271/7989
4272/7989
4273/7989
4274/7989
4275/7989
4276/7989
4277/7989
4278/7989
4279/7989
4280/7989
4281/7989
4282/7989
4283/7989
4284/7989
4285/7989
4286/7989
4287/7989
4288/7989
4289/7989
4290/7989
4291/7989
4292/7989
4293/7989
4294/7989
4295/7989
4296/7989
4297/7989
4298/7989
4299/7989
4300/7989
4301/7989
4302/7989
4303/7989
4304/7989
4305/7989
4306/7989
4307/7989
4308/7989
4309/7989
4310/7989
4311/7989
4312/7989
4313/7989
4314/7989
4315/7989
4316/7989
4317/7989
4318/7989
4319/7989
4320/7989
4321/7989
4322/7989
4323/7989
4324/7989
4325/7989
4326/7989
4327/7989
4328/7989
4329/7989
4330/7989


5457/7989
5458/7989
5459/7989
5460/7989
5461/7989
5462/7989
5463/7989
5464/7989
5465/7989
5466/7989
5467/7989
5468/7989
5469/7989
5470/7989
5471/7989
5472/7989
5473/7989
5474/7989
5475/7989
5476/7989
5477/7989
5478/7989
5479/7989
5480/7989
5481/7989
5482/7989
5483/7989
5484/7989
5485/7989
5486/7989
5487/7989
5488/7989
5489/7989
5490/7989
5491/7989
5492/7989
5493/7989
5494/7989
5495/7989
5496/7989
5497/7989
5498/7989
5499/7989
5500/7989
5501/7989
5502/7989
5503/7989
5504/7989
5505/7989
5506/7989
5507/7989
5508/7989
5509/7989
5510/7989
5511/7989
5512/7989
5513/7989
5514/7989
5515/7989
5516/7989
5517/7989
5518/7989
5519/7989
5520/7989
5521/7989
5522/7989
5523/7989
5524/7989
5525/7989
5526/7989
5527/7989
5528/7989
5529/7989
5530/7989
5531/7989
5532/7989
5533/7989
5534/7989
5535/7989
5536/7989
5537/7989
5538/7989
5539/7989
5540/7989
5541/7989
5542/7989
5543/7989
5544/7989
5545/7989
5546/7989
5547/7989
5548/7989
5549/7989
5550/7989
5551/7989
5552/7989
5553/7989
5554/7989
5555/7989
5556/7989


6650/7989
6651/7989
6652/7989
6653/7989
6654/7989
6655/7989
6656/7989
6657/7989
6658/7989
6659/7989
6660/7989
6661/7989
6662/7989
6663/7989
6664/7989
6665/7989
6666/7989
6667/7989
6668/7989
6669/7989
6670/7989
6671/7989
6672/7989
6673/7989
6674/7989
6675/7989
6676/7989
6677/7989
6678/7989
6679/7989
6680/7989
6681/7989
6682/7989
6683/7989
6684/7989
6685/7989
6686/7989
6687/7989
6688/7989
6689/7989
6690/7989
6691/7989
6692/7989
6693/7989
6694/7989
6695/7989
6696/7989
6697/7989
6698/7989
6699/7989
6700/7989
6701/7989
6702/7989
6703/7989
6704/7989
6705/7989
6706/7989
6707/7989
6708/7989
6709/7989
6710/7989
6711/7989
6712/7989
6713/7989
6714/7989
6715/7989
6716/7989
6717/7989
6718/7989
6719/7989
6720/7989
6721/7989
6722/7989
6723/7989
6724/7989
6725/7989
6726/7989
6727/7989
6728/7989
6729/7989
6730/7989
6731/7989
6732/7989
6733/7989
6734/7989
6735/7989
6736/7989
6737/7989
6738/7989
6739/7989
6740/7989
6741/7989
6742/7989
6743/7989
6744/7989
6745/7989
6746/7989
6747/7989
6748/7989
6749/7989


7844/7989
7845/7989
7846/7989
7847/7989
7848/7989
7849/7989
7850/7989
7851/7989
7852/7989
7853/7989
7854/7989
7855/7989
7856/7989
7857/7989
7858/7989
7859/7989
7860/7989
7861/7989
7862/7989
7863/7989
7864/7989
7865/7989
7866/7989
7867/7989
7868/7989
7869/7989
7870/7989
7871/7989
7872/7989
7873/7989
7874/7989
7875/7989
7876/7989
7877/7989
7878/7989
7879/7989
7880/7989
7881/7989
7882/7989
7883/7989
7884/7989
7885/7989
7886/7989
7887/7989
7888/7989
7889/7989
7890/7989
7891/7989
7892/7989
7893/7989
7894/7989
7895/7989
7896/7989
7897/7989
7898/7989
7899/7989
7900/7989
7901/7989
7902/7989
7903/7989
7904/7989
7905/7989
7906/7989
7907/7989
7908/7989
7909/7989
7910/7989
7911/7989
7912/7989
7913/7989
7914/7989
7915/7989
7916/7989
7917/7989
7918/7989
7919/7989
7920/7989
7921/7989
7922/7989
7923/7989
7924/7989
7925/7989
7926/7989
7927/7989
7928/7989
7929/7989
7930/7989
7931/7989
7932/7989
7933/7989
7934/7989
7935/7989
7936/7989
7937/7989
7938/7989
7939/7989
7940/7989
7941/7989
7942/7989
7943/7989


In [31]:
l = []
for y in y_val_inx:
    for x in y:
        l.append(x)

In [32]:
len(set(l))

27

In [220]:
y_test_inx = np_utils.to_categorical(y_test_inx)

In [221]:
y_train_inx = np_utils.to_categorical(y_train_inx)

In [222]:
y_val_inx = np_utils.to_categorical(y_val_inx)

# Model

In [36]:
model = keras.models.Sequential()
model

In [37]:
embeddings.shape

(56252, 300)

In [38]:
max_sentence_length

249

In [39]:
#https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
def embeddings_layer(max_length, embeddings, trainable=False, masking=False,
                     scale=False, normalize=False):

    vocab_size = embeddings.shape[0]
    embedding_size = embeddings.shape[1]

    _embedding = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_size,
        input_length=max_length, #if max_length > 0 else None,
        trainable=False,
        #mask_zero=masking, if max_length > 0 else False,
        weights=[embeddings]
    )

    return _embedding

In [40]:
len(tags_index)

26

In [217]:
from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

In [183]:
model = Sequential()
model.add(embeddings_layer(max_sentence_length, embeddings))
model.add(keras.layers.LSTM(300, return_sequences=True))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.LSTM(300, return_sequences=True))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(150, activation='relu'))
#model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(tags_index)+1, activation='softmax'))
#model.add(keras.layers.Activation('softmax'))

In [184]:
#from keras.utils.vis_utils import plot_model

In [185]:
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [186]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 249, 300)          16875600  
_________________________________________________________________
lstm_9 (LSTM)                (None, 249, 300)          721200    
_________________________________________________________________
lstm_10 (LSTM)               (None, 249, 300)          721200    
_________________________________________________________________
dense_13 (Dense)             (None, 249, 150)          45150     
_________________________________________________________________
dense_14 (Dense)             (None, 249, 50)           7550      
_________________________________________________________________
dropout_11 (Dropout)         (None, 249, 50)           0         
_________________________________________________________________
dense_15 (Dense)             (None, 249, 27)           1377      
Total para

## Compile

In [223]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy', 'categorical_accuracy', ignore_class_accuracy(0)])

In [224]:
history =  model.fit(
            X_train_inx[:800],
            y_train_inx[:800],
            batch_size = 128,
            epochs=2,
            validation_data = (X_val_inx[:100], y_val_inx[:100]),
            callbacks = [
                keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    min_delta=0.0001,
                    patience=10,
                    verbose=0,
                    mode='min'
                 )
            ]
        )

Train on 800 samples, validate on 100 samples
Epoch 1/2
800/800 [==============================] - 80s 100ms/step - loss: 0.3378 - acc: 0.9245 - categorical_accuracy: 0.9245 - ignore_accuracy: 0.0000e+00 - val_loss: 0.2998 - val_acc: 0.9225 - val_categorical_accuracy: 0.9225 - val_ignore_accuracy: 0.0000e+00
Epoch 2/2
800/800 [==============================] - 25s 31ms/step - loss: 0.3341 - acc: 0.9245 - categorical_accuracy: 0.9245 - ignore_accuracy: 0.0000e+00 - val_loss: 0.2979 - val_acc: 0.9225 - val_categorical_accuracy: 0.9225 - val_ignore_accuracy: 0.0000e+00


# fazer análises neste.... depois mandar pro servidor...

In [200]:
predicted_test = model.predict(X_test_inx)

In [201]:
t = []
for p in predicted_test:
    for c in p:
        t.append(np.argmax(c))
        

In [205]:
k = []
for p in y_test_inx:
    for c in p:
        k.append(np.argmax(c))

In [202]:
sorted(t, reverse=True)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [167]:
t = predicted_test[0][0]

In [168]:
max(t)

0.4401563

In [169]:
predicted_test.shape

(300, 249, 27)

In [91]:
predicted_test.shape

(300, 249, 27)

In [98]:
predicted_test[9].shape

(249, 27)

In [110]:
(np.all(predicted_test[9]))

True

In [114]:
predicted_test[0].shape

(249, 27)

In [133]:
predicted_test.shape[0]

300

In [146]:
teste = np.ones((predicted_test.shape[0], predicted_test.shape[1], predicted_test.shape[2]))

In [137]:
i = 0

j = 0

In [155]:
np.argmax(predicted_test[0][110])

0

In [147]:
teste[i][j] = (predicted_test[i][j] == max(predicted_test[i][j]))

IndexError: arrays used as indices must be of integer (or boolean) type

In [140]:
for i in predicted_test:
    for j in i:
        teste[i][j] = (predicted_test[i][j] == max(predicted_test[i][j]))


IndexError: arrays used as indices must be of integer (or boolean) type

In [123]:
predicted_test[0][101] == max(predicted_test[0][101])

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [111]:
predicted_test[9] = (predicted_test[8] ==  max(predicted_test[8]))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [74]:
predicted_test = (predicted_test == max(predicted_test))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [86]:
i = 0
for p in predicted_test:
    for b in p:
        i = i + 1
        b = (b == max(b))
        break
    break

In [87]:
b.shape

(27,)

In [88]:
b

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [89]:
t = (t == max(t))

In [90]:
t

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [60]:
X_test_inx.shape

(9987, 249)

In [61]:
y_test_inx.shape

(9987, 249, 27)

In [62]:
y_test_inx[0][3]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [63]:
predicted_test[0][3]

array([0.13757886, 0.03904151, 0.04679544, 0.0345195 , 0.04117398,
       0.03923594, 0.03097416, 0.02464281, 0.0417484 , 0.02912197,
       0.03302423, 0.0270728 , 0.03787771, 0.03424341, 0.03430784,
       0.02867949, 0.03172125, 0.03280784, 0.03111209, 0.03372767,
       0.02728003, 0.03019129, 0.03634657, 0.03472586, 0.03262221,
       0.02992634, 0.01950084], dtype=float32)

In [70]:
predicted_test.data.shape

(9987, 249, 27)

In [ ]:
tags_index

In [72]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences
 

In [74]:
print(logits_to_tokens(predicted_test, {i: t for t, i in tags_index.items()}))

KeyError: 0

In [78]:
for i, pred in enumerate(predicted_test[100]):
	try:
		print(sentence[i], ' : ', tags_index[np.argmax(pred)])
	except:
		pass
		# print('NA')

In [83]:
sorted(predicted_test[0][0])

[0.31801337,
 0.32218537,
 0.32299793,
 0.3276326,
 0.3278319,
 0.328435,
 0.33004722,
 0.33046103,
 0.3323479,
 0.3344832,
 0.33542857,
 0.33555093,
 0.3381184,
 0.3394882,
 0.3413541,
 0.34203148,
 0.34526813,
 0.3500948,
 0.35055625,
 0.3636256,
 0.37206104,
 0.3778393,
 0.38035557,
 0.40135187,
 0.40262803,
 0.42853042,
 0.731753]

In [62]:
metrics = sklearn.metrics.precision_recall_fscore_support(y_test_inx, predicted_test)
#pd.DataFrame(list(metrics), index=['Precision', 'Recall', "F-Score", "Support"])

ValueError: unknown is not supported

In [168]:
sklearn.metrics.confusion_matrix(y_true=y_test_inx[:300], y_pred=predicted_test)

ValueError: unknown is not supported

In [170]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(y_true=y_test, y_score=predicted_test)
sklearn.metrics.auc(fpr, tpr)

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead.

In [185]:
y_train_inx.shape

(31956, 249, 27)

In [188]:
y_test_inx#[0][0]

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0.

In [178]:
t = y_train_inx[0][0]

In [ ]:
t

In [179]:
sklearn.metrics.confusion_matrix(y_train_inx[0][0], t)

array([[26,  0],
       [ 0,  1]])

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [419]:
predicted_test = model.predict(X=test_set['text_tokens'])
sklearn.metrics.confusion_matrix(y_true=test_set['tags'], y_pred=predicted_test)

TypeError: predict() got an unexpected keyword argument 'X'

grafico do loss
curva ROC